In [5]:
# get all season 11 reports. landing page https://sites.google.com/a/marssociety.org/mdrs/home/field-reports
# need to get crews 108-118, as crew104 was the last crew in season 10

from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import csv
import urllib.request
import pandas as pd
import re

In [3]:
#need to follow 12 links from the landing page. Easier to do it by hand than to scrape from the landing page. 


crews = ['crew108','crew109', 'crew110a', 'crew110b', 'crew111', 'crew112', 'crew113', 
         'crew114'] #other crews either have no daily reports or erroneaous report links that lead to wrong reports (e.g. 115 and 116 reports links to crew 97 reports)

crew_links = []

for crew in crews:
    full_link = 'https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/' + crew
    crew_links.append(full_link)







In [6]:

#get all links for all crews

day_links = [] 

for link in crew_links:
    print(link)
    url = urllib.request.urlopen(link)
    soup = bs(url, 'lxml')
    results = soup.find_all('a')
    for x in results:
           if 'day' in x['href'] and x.has_attr('dir'): # get full links to daily reports
                d_link = link + "/" + x.get_text() 
                print(d_link)
                day_links.append(d_link)




# get full links:


#'/a/marssociety.org/mdrs/home/field-reports/crew108/day01'.split('/', 3)

#https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day01

day_links = [x.lower() for x in day_links] #making sure links are lower case only






https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/Day01
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day02
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day03
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day04
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day05
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day06
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day07
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day08
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day09
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day10
https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day11
https://sites.google.com/a/marssoc

In [59]:
#parsing links to days 

crew = []
title =[]
text = []
html = []
urls = []
days = []
pattern = "Date and time: (.*?)<"

# parsing day_link
#day = 'https://sites.google.com/a/marssociety.org/mdrs/home/field-reports/crew108/day01'


for d in day_links:  
    try:   #this is the list of all crew/dayX links  that contains reports from dayX
        l = urllib.request.urlopen(d).read()
        soup = bs(l, 'lxml')
        for x in soup.find_all("div", {"class": "sites-embed-content-textbox"}):
            html.append(x) #html text of the report
            crew.append(d.rsplit('/',2)[-2])
            title.append(x.h2.get_text())
            text.append (x.get_text())
            urls.append(d)
            days.append(d.rsplit('/',1)[-1]) 
    except urllib.error.HTTPError as exception:
        print(d, exception)


print(len(crew), len(title), len(text),  len(urls), len(days))   






    

353 353 353 0 353 353


In [ ]:
datetimebr = "Date and time: [<br/>]*(.*?)<"
date = "Date: (.*?)<"
datetime = "Date and time: (.*?)<"


In [60]:
df = pd.DataFrame({'crew': crew, 'crewday': days, 'header': title, 'body':text, 'html': html, 'link': urls})


crew_n = []
for x in df['crew']:
    c = x.rsplit("crew")
    crew_n.append(c[1])

df['crew'] = crew_n


df_clean = df[df['header'] != df['body']]

len(df_clean), len(df)


df_clean.to_csv('s11_reports.csv')